In [1]:
import os, pickle, random
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter

In [2]:
os.chdir('..')

In [3]:
from src.models.genetic_algorithm import *
from src.models.train_animation_predictor import *
from src.models import config
from src.models.ordinal_classifier_fnn import *
from src.models.genetic_algorithm import create_animation_vector
from src.animations.insert_animation import create_animated_svg
from src.models.entmoot_functions import *

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Load test data for evaluation

In [5]:
test_data = pd.read_csv('data/path_selector/path_selector_test.csv')

In [6]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg
0,logo_147,10,0.288970,-7.284798,1.094182,-4.437811,-0.352077,-0.933593,-0.200254,0.931484,0.506142,0.100814,0,1.0,1.0,157,157,156,157,157,156,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,0.017877,0.111550,0.168001,0.084745,13
1,logo_147,0,-2.807291,-11.223132,3.151704,2.541029,-0.119346,0.909742,2.923519,-1.477561,0.059558,-1.025471,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.077887,0.707306,0.038944,0.562185,13
2,logo_147,1,0.869143,-11.525720,-0.201577,3.906369,-2.661400,-0.757390,-0.195925,-0.380663,-0.033850,0.286074,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.026691,0.467979,0.161487,0.450085,13
3,logo_147,12,9.519546,-1.598271,-4.577534,-2.522939,-0.754545,-0.054362,-1.478206,-1.002636,-1.703539,-0.243925,0,1.0,1.0,22,65,148,22,65,148,136.230769,-114.230769,142.846154,-77.846154,154.769231,-6.769231,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,0.057923,0.271332,0.502114,0.135666,13
4,logo_147,2,-0.253583,-11.400860,-0.170891,5.689338,-0.972701,1.972220,-0.119872,0.083583,0.253451,-0.066204,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.050151,0.477448,0.113555,0.445340,13


In [7]:
test_data = retrieve_m1_predictions(test_data)

2021-05-10 10:26:48::INFO::Number of animated paths: 448/1892


In [8]:
test_data = retrieve_animation_midpoints(test_data, drop=True)

2021-05-10 10:26:48::INFO::Non-animated paths won't be used for training
2021-05-10 10:26:48::INFO::Start extraction midpoint of animated paths as feature


In [9]:
scaler = pickle.load(open(config.scaler_path, 'rb'))
test_data[config.sm_features] = scaler.transform(test_data[config.sm_features])

In [10]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations
0,logo_147,10,-0.611706,-1.436978,0.377512,-0.996121,0.386014,-1.090409,-0.385924,0.365459,0.306280,0.053203,0,1.0,1.0,0.024062,0.320447,0.462987,157,157,156,0.224118,-0.153866,0.436991,-0.009219,0.680529,-0.035167,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,-1.036201,-0.835023,-1.035631,-1.715457,-0.583856,1,"[10, 12, 11]","(0.0428012672222377, 0.08474520367214555)",-1.873210,-1.851513
1,logo_147,12,0.578140,-0.445076,-0.727123,-0.615888,0.241741,-0.754951,-1.201539,-0.488212,-0.989006,-0.228000,0,1.0,1.0,-1.286856,-0.732134,0.375291,22,65,148,0.224118,-1.579276,0.436991,-1.153619,0.680529,-0.136736,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.862178,-0.244325,0.393178,-1.458720,-0.583856,1,"[10, 12, 11]","(0.923646143983946, 0.13566598867571875)",1.667780,-1.623868
2,logo_147,11,0.463658,-1.014969,-0.523243,1.329473,0.307836,-0.318510,-1.637074,-0.475066,-0.652260,-0.724454,0,1.0,1.0,-1.286856,-0.732134,0.375291,22,65,148,0.224118,-1.579276,0.436991,-1.153619,0.680529,-0.136736,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.326326,1.708559,0.017942,0.883455,-0.583856,1,"[10, 12, 11]","(0.6923176974460455, 0.6002088945082733)",0.737841,0.452903
3,logo_148,11,-0.797719,-1.730072,0.653065,0.788841,-0.354336,-1.763647,-0.792142,0.582797,-1.524141,-0.046400,0,1.0,1.0,0.024062,0.320447,0.462987,0,0,0,-1.023876,0.848620,-0.546496,0.795634,0.595453,0.036265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.027950,0.869430,-1.219746,-0.699239,-0.528969,1,[11],"(0.4999999999999999, 0.5)",-0.035274,0.004912
4,logo_149,11,0.110555,0.294885,-0.924922,0.831050,-0.864547,-1.692691,-1.349850,1.371332,-2.464679,-0.768306,0,1.0,1.0,0.024062,0.320447,0.462987,0,0,0,-1.023876,0.848620,-0.546496,0.795634,0.595453,0.036265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.512969,0.043825,0.386242,-1.262228,-0.528969,1,"[11, 12, 13]","(0.650810561967709, 0.2862981256047344)",0.570983,-0.950457


In [11]:
path_vectors = torch.tensor(test_data[config.sm_features].to_numpy(), dtype=torch.float)

## 2. Apply animation predictor model to test data

In [12]:
ap = AnimationPredictor()
ap.load_state_dict(torch.load(config.ap_state_dict_path))

<All keys matched successfully>

In [14]:
animation_predictions = ap(path_vectors)

In [15]:
animation_vectors = [create_animation_vector(animation_predictions[i]).tolist() for i in range(animation_predictions.shape[0])]
test_data['animation_vector'] = animation_vectors

In [16]:
rewards = predict(prepare_sm_input(path_vectors, animation_predictions))
test_data['reward'] = rewards

In [17]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations,animation_vector,reward
0,logo_147,10,-0.611706,-1.436978,0.377512,-0.996121,0.386014,-1.090409,-0.385924,0.365459,0.306280,0.053203,0,1.0,1.0,0.024062,0.320447,0.462987,157,157,156,0.224118,-0.153866,0.436991,-0.009219,0.680529,-0.035167,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,-1.036201,-0.835023,-1.035631,-1.715457,-0.583856,1,"[10, 12, 11]","(0.0428012672222377, 0.08474520367214555)",-1.873210,-1.851513,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",2
1,logo_147,12,0.578140,-0.445076,-0.727123,-0.615888,0.241741,-0.754951,-1.201539,-0.488212,-0.989006,-0.228000,0,1.0,1.0,-1.286856,-0.732134,0.375291,22,65,148,0.224118,-1.579276,0.436991,-1.153619,0.680529,-0.136736,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.862178,-0.244325,0.393178,-1.458720,-0.583856,1,"[10, 12, 11]","(0.923646143983946, 0.13566598867571875)",1.667780,-1.623868,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",2
2,logo_147,11,0.463658,-1.014969,-0.523243,1.329473,0.307836,-0.318510,-1.637074,-0.475066,-0.652260,-0.724454,0,1.0,1.0,-1.286856,-0.732134,0.375291,22,65,148,0.224118,-1.579276,0.436991,-1.153619,0.680529,-0.136736,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.326326,1.708559,0.017942,0.883455,-0.583856,1,"[10, 12, 11]","(0.6923176974460455, 0.6002088945082733)",0.737841,0.452903,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",2
3,logo_148,11,-0.797719,-1.730072,0.653065,0.788841,-0.354336,-1.763647,-0.792142,0.582797,-1.524141,-0.046400,0,1.0,1.0,0.024062,0.320447,0.462987,0,0,0,-1.023876,0.848620,-0.546496,0.795634,0.595453,0.036265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.027950,0.869430,-1.219746,-0.699239,-0.528969,1,[11],"(0.4999999999999999, 0.5)",-0.035274,0.004912,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",2
4,logo_149,11,0.110555,0.294885,-0.924922,0.831050,-0.864547,-1.692691,-1.349850,1.371332,-2.464679,-0.768306,0,1.0,1.0,0.024062,0.320447,0.462987,0,0,0,-1.023876,0.848620,-0.546496,0.795634,0.595453,0.036265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.512969,0.043825,0.386242,-1.262228,-0.528969,1,"[11, 12, 13]","(0.650810561967709, 0.2862981256047344)",0.570983,-0.950457,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",3


In [18]:
avg_reward = test_data['reward'].mean()
reward_distribution = dict(test_data['reward'].value_counts())

In [19]:
print(f'Average evaluation score using the genetic algorithm: {avg_reward :.4f}')
print(f'Distribution of evaluation scores: {reward_distribution}')

Average evaluation score using the genetic algorithm: 2.2746
Distribution of evaluation scores: {2: 260, 3: 154, 1: 33, 4: 1}
